In [1]:
import csv
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
import skimage
from skimage.io import ImageCollection,concatenate_images,imread
from skimage.transform import resize
from skimage.color import gray2rgb
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from keras.layers import Flatten, Dense, Dropout
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from keras.applications.resnet50 import decode_predictions
#from keras.applications.xception import Xception
from keras.applications import resnet50
from keras.applications import xception
from keras.applications import vgg16
from sklearn.ensemble import VotingClassifier

#import mahotas as mt
#import xgboost as xg




Using TensorFlow backend.


In [2]:


#utility functions for pickling 
# dumping into pickle
def pickleDump(dataToBeDumoped,name):
    import pickle 
    dbfile = open(name, 'ab') 
    pickle.dump(dataToBeDumoped,dbfile)
    dbfile.close()

    
#loadingPickle
def loadPickleData(nameOfPickleFile):
    import pickle 
    dbfile = open(nameOfPickleFile, 'rb')      
    loaded = pickle.load(dbfile)
    
    return loaded
#extract haralick from pickle file
def extractHaralickPickle(path):
    return loadPickleData(path+'haralick')
    

In [3]:

def returnImagesAndLabels(path):
    
    lab = pd.read_csv('dataset/train.csv')
    images = []
    files = []
    labels = []
    for index, row in lab.iterrows():
        files.append(row['Id'])
        labels.append(row['Category'])
    target = []
    #img_c = cv2.imread(path+ '/'+ files[i])
    #img_c = cv2.cvtColor(img_c,cv2.COLOR_BGR2HSV)
    
    #cv2.normalize(histr,histr)
    #fd = histr.flatten()
    #hist.append(fd)
    for i in range(len(files)):
        name = path+'/'+files[i]
        overallPath = path+'train/'+files[i]
        img = cv2.imread(path+ 'train/'+ files[i],0)
        #histr = cv2.calcHist([img],[0,1,2],None,[8,8,8],[0,256,0,256,0,256]) 
        images.append(img)
        
        target.append(labels[i])
        
    return images,target
        
#extracting images and their target/labels
images, target = returnImagesAndLabels('dataset/')

#converting them to numpy arrays
images = np.array(images)
target = np.array(target)


def hogFeatures(pixels):
    from skimage.feature import hog
    hogs = []
    for i in range(len(pixels)):
        if(pixels[i].shape == 3):
            fd = hog(pixels[i], orientations=16, pixels_per_cell=(16, 16),cells_per_block=(2, 2),multichannel=True)
            hogs.append(fd)
        else:
            fd = hog(pixels[i], orientations=16, pixels_per_cell=(16, 16),cells_per_block=(2, 2))
            hogs.append(fd)
    return hogs


#pass images as arguements
hog = hogFeatures(images)
hog = np.array(hog)
print(hog.shape)



haralick = extractHaralickPickle('')


(10000, 576)


In [4]:
hog_haralick = np.concatenate((hog,haralick),axis=1)

In [5]:
def getHuMoments(path):
    return loadPickleData(path+'humom')

In [6]:
humom = getHuMoments('')

In [7]:
hog_humom_haralick = np.concatenate((humom,hog,haralick),axis=1)

In [8]:
def ensemble(X_train,y_train,X_test,y_test):
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import GaussianNB
    from sklearn.ensemble import RandomForestClassifier, VotingClassifier
    clf1 = LogisticRegression( random_state=1)
    clf2 = RandomForestClassifier(n_estimators=1000, random_state=1)
    clf3 = GaussianNB()    
    model = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],voting='soft')
    model = model.fit(X_train, y_train)
    print(model.score(X_test,y_test))
        
    
    
    '''avoid run NN and svm, doesnt affect the result much, but affect the run time'''
    
    '''
    from sklearn.neural_network import MLPClassifier
    nn = MLPClassifier()
    
    nn.fit(X_train,y_train)
    nn.predict(X_test,y_train)
    
    from sklearn.svm import SVC
    svm = SVC(kernel = 'linear')
    svm.fit(X_train,y_train)
    svm.predict(X_test,y_train)
    '''
#    final_pred = stats.mode(predictions,axis=1)

'''
def ensemble_result(X_train,y_train,X_test,y_test):
    
    preds = ensemble(X_train,y_train,X_test,y_test)
    final_preds = []
    for i in range(pred.shape[0]):
        mode = preds[i][j]
        for j in range(pred.shape[1]):
            
            
        

        
        
    '''

'\ndef ensemble_result(X_train,y_train,X_test,y_test):\n    \n    preds = ensemble(X_train,y_train,X_test,y_test)\n    final_preds = []\n    for i in range(pred.shape[0]):\n        mode = preds[i][j]\n        for j in range(pred.shape[1]):\n            \n            \n        \n\n        \n        \n    '

In [9]:
hog_humom_haralick = np.concatenate((humom,hog,haralick),axis=1)
X_train, X_test, y_train, y_test = train_test_split(hog_humom_haralick, target, test_size=0.20, random_state=42)



In [10]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((8000, 596), (8000,), (2000, 596), (2000,))

In [14]:
ensemble(X_train,y_train,X_test,y_test)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.249
